In [18]:
# import os
import sys
sys.path.append("../src/")
# import argparse
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error

from utils import *
from preprocess import PreProcessor
from feature_engineering import make_feature
from model import ClassificationModel, output_index, RegressionModel, OptunaProcessor

In [19]:
classification_method = 'False'
log_transform = 'False'
regression_model_name = 'xgboost'
pickle_path='../pickle'
version='version4'
method='replace_areamean_no_cls_no_log'

In [20]:
## load data set
X_train, y_train, X_valid, y_valid = load_dataset(mode='train')
categorical_feature = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG', 'BREADTH', 'DEPTH', 'DRAUGHT', 'year']
minmaxscale_feature = ['DIST', 'BUILT', 'DEADWEIGHT', 'GT', 'LENGTH', 'DUBAI', 'BRENT', 'WTI', 'BDI_ADJ', 'PORT_SIZE']

In [22]:
X_train.columns

Index(['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'ATA', 'ID',
       'BREADTH', 'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT', 'LENGTH',
       'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN',
       'ATA_LT', 'DUBAI', 'BRENT', 'WTI', 'BDI_ADJ', 'PORT_SIZE'],
      dtype='object')

* onehotencoding : ARI_CO, SHIP_TYPE_CATEGORY, BUILT, FLAG
* pca : 

In [37]:
by_ID = X_train.groupby(by='ID')
by_ID['GT'].value_counts()
print('ID', len(X_train.ID.unique()))
print('BREADTH', len(X_train.BREADTH.unique()))
print('BUILT', len(X_train.BUILT.unique()))
print('DEADWEIGHT', len(X_train.DEADWEIGHT.unique()))
print('DRAUGHT', len(X_train.DRAUGHT.unique()))
print('GT', len(X_train.GT.unique()))
print('LENGTH', len(X_train.LENGTH.unique()))
print('SHIPMANAGER', len(X_train.SHIPMANAGER.unique()))
print('FLAG', len(X_train.FLAG.unique()))

ID 23385
BREADTH 6
BUILT 62
DEADWEIGHT 1947
DRAUGHT 3
GT 1757
LENGTH 37
SHIPMANAGER 6202
FLAG 122


In [85]:
X_train.ID.unique()

array(['U512824', 'C234752', 'L383478', ..., 'P146816', 'U837747',
       'K221481'], dtype=object)

In [86]:
X_train.loc[X_train["ID"] == 'U512824']

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,...,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE
0,PE,HYV6,Bulk,0.000000,2016-11-02 20:11,U512824,30.0,13,32600,10.0,...,NaN,NaN,NaN,NaN,15,44.08,46.86,45.34,808.820730,0.000010
28552,CN,EKP8,Bulk,0.000000,2017-01-12 13:02,U512824,30.0,13,32600,10.0,...,NaN,NaN,NaN,NaN,21,53.45,56.01,53.01,859.416578,0.001660
44358,IN,UJM2,Bulk,27.662068,2021-03-06 3:05,U512824,30.0,13,32600,10.0,...,3.90,1.08,23.8,2.861554,8,68.32,68.24,65.05,1641.794345,0.000217
55501,TW,JWI3,Bulk,0.000000,2017-02-02 15:04,U512824,30.0,13,32600,10.0,...,NaN,NaN,NaN,NaN,23,54.94,56.56,53.54,740.693385,0.000990
74917,RU,VXM8,Bulk,15.419678,2021-07-05 2:14,U512824,30.0,13,32600,10.0,...,-0.91,3.73,27.0,2.762940,4,75.88,74.53,73.37,2745.660577,0.000145
88087,CN,EKP8,Bulk,27.309051,2017-06-14 4:03,U512824,30.0,13,32600,10.0,...,NaN,NaN,NaN,NaN,12,46.84,47.00,44.73,831.542596,0.001660
99996,BR,VFD8,Bulk,8.864219,2015-05-30 10:45,U512824,30.0,13,32600,10.0,...,NaN,NaN,NaN,NaN,8,55.75,57.51,50.91,994.948079,0.000061


In [67]:
len(X_train)

257208

In [ ]:
data = pd.concat([X_train, y_train], axis=1)
# data[minmaxscale_feature + ['CI_HOUR']].corr()

In [69]:
import seaborn as sns
# sns.scatterplot(y='CI_HOUR', x='ARI_CO', data=data)
# sns.scatterplot(y='CI_HOUR', x='BUILT', data=data)
# sns.scatterplot(y='CI_HOUR', x='DEADWEIGHT', data=data)
# sns.scatterplot(y='CI_HOUR', x='DEPTH', data=data)
# sns.scatterplot(y='CI_HOUR', x='GT', data=data)
# sns.scatterplot(y='CI_HOUR', x='LENGTH', data=data)
# sns.scatterplot(y='CI_HOUR', x='SHIPMANAGER', data=data)
# sns.scatterplot(y='CI_HOUR', x='FLAG', data=data)

In [83]:
# X_train.groupby('ID')['BREADTH'].value_counts().to_frame()
# X_train.groupby('ID')['DEADWEIGHT'].value_counts().to_frame()
# X_train.groupby('ID')['DEPTH'].value_counts().to_frame()
# X_train.groupby('ID')['DRAUGHT'].value_counts().to_frame()
# X_train.groupby('ID')['GT'].value_counts().to_frame()
# X_train.groupby('ID')['LENGTH'].value_counts().to_frame()
# X_train.groupby('ID')['SHIPMANAGER'].value_counts().to_frame()
# X_train.groupby('ID')['FLAG'].value_counts().to_frame()


In [40]:
X_train.DRAUGHT.unique()

array([10., 20.,  0.])

In [4]:
## preprocess data set
preprocessing = PreProcessor(categorical_feature=categorical_feature, minmaxscale_feature=minmaxscale_feature)
mean_values_train = preprocessing.nan_area_mean_fit(X_train)
ship_mean_values_train = preprocessing.nan_ship_fit(X_train)

X_train = preprocessing.preprocess(X_train, method='area_mean', mean_values=mean_values_train, ship_mean_values=ship_mean_values_train)
X_valid = preprocessing.preprocess(X_valid, method='area_mean', mean_values=mean_values_train, ship_mean_values=ship_mean_values_train)

encoder_dict = preprocessing.categorical_process_fit(X_train)
scaler = preprocessing.minmaxscale_process_fit(X_train)

X_train = preprocessing.transform(X_train, encoder=encoder_dict, scaler=scaler)
X_valid = preprocessing.transform(X_valid, encoder=encoder_dict, scaler=scaler)

X_train, y_train = reset_data(X_train, y_train)
X_valid, y_valid = reset_data(X_valid, y_valid)

In [5]:
## feature engineering
X_train = make_feature(X_train)    
X_valid = make_feature(X_valid)
print("Done Feature Engineering!")

Done Feature Engineering!


In [6]:
## binary
if classification_method == 'True':
    extractor = ClassificationModel(
        X_train=X_train, 
        y_train=y_train, 
        X_valid=X_valid, 
        y_valid=y_valid, 
        classifier_name=classification_model_name,         
        ) 
    classifier = extractor.fit() 
        
    train_none_zero_index = output_index(classifier=classifier, df=X_train)
    valid_none_zero_index = output_index(classifier=classifier, df=X_valid)
    X_train_ = X_train.loc[train_none_zero_index,:]
    y_train_ = y_train.loc[train_none_zero_index]
    X_valid_ = X_valid.loc[valid_none_zero_index,:]
    y_valid_ = y_valid.loc[valid_none_zero_index]
else:
    X_train_ = X_train.copy()
    y_train_ = y_train.copy()
    X_valid_ = X_valid.copy()
    y_valid_ = y_valid.copy()

In [7]:
## log transform
if log_transform == 'True':
    y_train_ = np.log1p(y_train_)
    y_valid_ = np.log1p(y_valid_)        

In [8]:
# ## progress optuna
# if args.mode == 'optuna':
#     print("     Strat Optuna!")
#     optuna = OptunaProcessor(
#         X_train=X_train_, 
#         y_train=y_train_, 
#         X_valid=X_valid_, 
#         y_valid=y_valid_, 
#         categorical_feature=categorical_feature
#         )
#     optuna_process = optuna.run_optuna(n_trials=args.n_trials, model_name=args.regression_model_name)
#     best_optuna_params = optuna_process.best_trial.params
#     print("     Done Optuna!")
    
#     save_pickle(file=best_optuna_params, file_name=f"{args.regression_model_name}_optuna", path=f"{args.pickle_path}/{args.version}/{args.method}/regressor")

# else: 
best_optuna_params = load_pickle(file_name=f"{regression_model_name}_optuna", path=f"{pickle_path}/{version}/{method}/regressor")

In [9]:
## train model
model = RegressionModel(
    model_params=best_optuna_params, 
    categorical_feature=categorical_feature, 
    X_train=X_train_, 
    y_train=y_train_, 
    X_valid=X_valid_, 
    y_valid=y_valid_, 
    model_name=regression_model_name
    )
    
model = model.fit()

model_feature_importance = pd.DataFrame(
    {'feature' : model.feature_names_in_, 
     'importance' : model.feature_importances_}).sort_values(by='importance', ascending=False)

# save_pickle(file=model, file_name=args.regression_model_name, path=f"{args.pickle_path}/{args.version}/{args.method}/regressor")

In [17]:
model_feature_importance

,feature,importance
0,DIST,0.231901
1,ID,0.106151
2,SHIPMANAGER,0.103900
3,SHIP_TYPE_CATEGORY,0.087586
4,ARI_PO,0.061279
5,FLAG,0.026049
9,DEADWEIGHT,0.022324
8,GT,0.022144
12,WTI,0.021813
10,BDI_ADJ,0.021761


In [ ]:
model = RegressionModel(
    model_params=best_optuna_params, 
    categorical_feature=categorical_feature, 
    X_train=X_train_[model_feature_importance.loc[model_feature_importance['importance'] > 0, 'feature'].tolist()], 
    y_train=y_train_, 
    X_valid=X_valid_[model_feature_importance.loc[model_feature_importance['importance'] > 0, 'feature'].tolist()], 
    y_valid=y_valid_, 
    model_name=regression_model_name
    )
    
model = model.fit()


if classification_method == 'True':    
    y_valid_pred = y_valid.copy()
    y_valid_pred['CI_HOUR'] = 0
    y_valid_pred.loc[valid_none_zero_index] = model.predict(X_valid.loc[valid_none_zero_index,:])
else: 
    y_valid_pred = model.predict(X_valid_[model_feature_importance.loc[model_feature_importance['importance'] > 0, 'feature'].tolist()])
    
if log_transform == 'True':
    y_valid_pred = np.expm1(y_valid_pred)

if classification_method == 'False':
    y_valid_pred = after_process(y_pred=y_valid_pred)    

mae = mean_absolute_error(y_true=y_valid, y_pred=y_valid_pred)

In [10]:
print(mae)

50.54406052678277


In [11]:
model_feature_importance = pd.DataFrame(
    {'feature' : model.feature_names_in_, 
     'importance' : model.feature_importances_}).sort_values(by='importance', ascending=False)

In [15]:
len(model_feature_importance)

25

In [16]:
model_feature_importance

,feature,importance
0,DIST,0.231901
1,ID,0.106151
2,SHIPMANAGER,0.103900
3,SHIP_TYPE_CATEGORY,0.087586
4,ARI_PO,0.061279
5,FLAG,0.026049
9,DEADWEIGHT,0.022324
8,GT,0.022144
12,WTI,0.021813
10,BDI_ADJ,0.021761


In [13]:
len(model_feature_importance.loc[model_feature_importance['importance'] > 0, 'feature'])

25

In [14]:
model_feature_importance.loc[model_feature_importance['importance'] > 0, 'feature'].tolist()

['DIST',
 'ID',
 'SHIPMANAGER',
 'SHIP_TYPE_CATEGORY',
 'ARI_PO',
 'FLAG',
 'DEADWEIGHT',
 'GT',
 'WTI',
 'BDI_ADJ',
 'wind_speed',
 'U_WIND',
 'LENGTH',
 'ARI_CO',
 'DUBAI',
 'ATA_LT',
 'BRENT',
 'V_WIND',
 'BN',
 'day',
 'hour',
 'month',
 'BUILT',
 'AIR_TEMPERATURE',
 'PORT_SIZE']